In [1]:
# Load the necessary package
library(readr)

# Specify the file path
file_path <- "../data/geog_by_month.csv"

# Read the CSV file
data <- read_csv(file_path)

# Load the necessary packages
library(ggplot2)

New names:
• `` -> `...1`
Rows: 82845 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): Month, Country, State/Region, Geography Type, Geography Name
dbl (7): ...1, Year, Year-Month, Geography Code, Percent, Percent_3ma, N

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [2]:
# check the unique values in the 'Country' column
# keep only US counties
data <- data[data$Country == "USA",]

In [3]:
# collapse the data by year and county
# first multiply N by Percent to arrive at the number of WFH jobs
# second we add up the number of WFH jobs for each year
# third we devide the number of WFH jobs for each year by the total
# number of jobs for that year to get the percentage of WFH jobs

library(dplyr)

# Assuming your data frame is named 'data' and it has columns 'Year', 'N', and 'Percent' # nolint
data_by_year_county <- data %>% # nolint
  group_by(`Year`,`Geography Code`,`State/Region`,`Geography Name`) %>% # nolint
  mutate(WFH_jobs = N * Percent / 100) %>%
  summarise(Total_WFH_jobs = sum(WFH_jobs),
            Total_jobs = sum(N),
            Percent_WFH_jobs = Total_WFH_jobs / Total_jobs * 100)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




`summarise()` has grouped output by 'Year', 'Geography Code', 'State/Region'.
You can override using the `.groups` argument.


In [4]:
library(tigris)
library(sf)

To enable caching of data, set `options(tigris_use_cache = TRUE)`
in your R script or .Rprofile.

Linking to GEOS 3.12.1, GDAL 3.7.3, PROJ 9.2.1; sf_use_s2() is TRUE



In [5]:
# Get the shapefiles for US counties
us_counties <- counties()
# Filter out non-mainland states and territories
mainland_us_counties <- us_counties %>%
  filter(!(STATEFP %in% c("02", "15", "60", "66", "69", "72", "78")))

# STATEFP codes:
# 02 - Alaska
# 15 - Hawaii
# 60 - American Samoa
# 66 - Guam
# 69 - Northern Mariana Islands
# 72 - Puerto Rico
# 78 - Virgin Islands

Retrieving data for the year 2022



  |======================================================================| 100%


Warning message in CPL_read_ogr(dsn, layer, query, as.character(options), quiet, :
“GDAL Message 1: /tmp/RtmpXmnNYJ/tl_2022_us_county.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.”


In [6]:
# Convert the Geography Code to a character so it can be merged with the shapefiles
# fill the four digit code with leading zeros
data_by_year_county$'Geography Code' <- sprintf("%05d", data_by_year_county$'Geography Code')
data_by_year_county$'Geography Code' <- as.character(data_by_year_county$'Geography Code')

In [7]:
# for each year, merge the data with the shapefiles
for (year in unique(data_by_year_county$Year)) {
  data_by_year_county_year <- data_by_year_county[data_by_year_county$Year == year,]
  us_counties_year <- merge(mainland_us_counties, data_by_year_county_year,
   all.x = TRUE, by.x = "GEOID", by.y = "Geography Code")
  # fill zero or missing with 0.0001 to avoid log(0) error
  us_counties_year$Percent_WFH_jobs[is.na(us_counties_year$Percent_WFH_jobs)] <- 0.0001
  us_counties_year$Percent_WFH_jobs[us_counties_year$Percent_WFH_jobs==0] <- 0.0001


  # Create the map
  p <- ggplot() +
  geom_sf(data = us_counties_year, aes(fill = Percent_WFH_jobs, geometry = geometry)) +
  scale_fill_gradient(low = "white", high = "red", trans = 'log') +
  theme_minimal() +
  labs(fill = "Percent WFH Jobs", title = "Heatmap of Percent WFH Jobs by County")
  # Save the map by year automatically
  ggsave(paste0("../output_blogpost_1/figure3_heatmap_", year, ".png"), width = 16, height = 10, dpi = 300, plot = p, device = "png", bg = "white")

}